드라이브 마운트 확인

소스코드 참조

 https://www.kaggle.com/sharmilaupadhyaya/20newsgroup-classification-using-keras-bert-in-gpu

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
path = "/content/drive/My Drive/2020 1학기/자연어처리/friends"
print(os.listdir(path))

['en_data.csv', 'en_sample.csv', 'friends_train.json', 'friends_dev.json', 'friends_test.json', '.ipynb_checkpoints', 'model.h5', 'model_by_label.h5', 'en_bert_results.csv', 'en_bert_results_round.csv', 'en_results.csv', 'en_results_round.csv', 'en_results_6.csv', 'en_results_round_6.csv', 'en_results_9.csv', 'en_results_round_9.csv', 'en_results_round_3.csv', 'en_results_3.csv', 'lstm_2_model.h5', 'lstm_2_model_25.h5', 'nlp_en_cnn_3.csv', 'nlp_en_cnn_6.csv', 'nlp_en_cnn_9.csv', 'nlp_en_Dual_Encoder_LSTM_1.csv']


임포트

In [ ]:
import numpy as np
import pandas as pd
import json

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model, load_model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


전처리

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

# text 를 전처리 해서 쓰려고 만들었는데 막상 적용해보니 성능이 떨어져서 비활성화함
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

# 대략적인 전처리
def preprocess_dataframe(df):
  df = df.dropna() # none 제거해줌

  for i in df.index:
    i_utterance = df.at[i, 'i_utterance']
    i_dialog = df.at[i, 'i_dialog']
    utterance = df.at[i, 'utterance']

    if i_utterance == 0: # 새로운 dialog 시작되면
      context = "" #초기화
    else:
      context = context + " " + utterance
    
    # 한 dialog 의 텍스트를 다 붙여놓은 형태. 이전 대화의 context도 포함하기 위함
    df.at[i, 'context'] = context
    
  # 길이 
  df['l'] = df['context'].apply(lambda x: len(str(x).split(' ')))
  print("mean length of sentence: " + str(df.l.mean()))
  print("max length of sentence: " + str(df.l.max()))
  print("std dev length of sentence: " + str(df.l.std()))
  return df

# json 파일을 읽어서, kaggle sample csv를 읽은 것처럼 포맷을 정리해준다
def read_json_make_dataframe(name='train'):  
  file = open('%s/friends_%s.json' % (path, name), encoding='latin1')
  d = json.load(file)
  data = []
  columns = ['i_dialog', 'i_utterance', 'speaker', 'utterance', 'annotation', 'emotion']
  i_dialog = -1
  for dialog in d:
    i_dialog += 1 # 0부터 시작
    i_utterance = -1
    for row in dialog:
      i_utterance += 1 # 0부터 시작
      data.append([
                   i_dialog, 
                   i_utterance, 
                   row['speaker'], 
                   row['utterance'], 
                   str(row['annotation']), 
                   row['emotion']
                   ])
      
        

  df = pd.DataFrame(data, columns=columns)
  df = preprocess_dataframe(df)
  return df

# kaggle data set 을 읽기 위한 함수
def read_csv_make_dataframe():
  df = pd.read_csv('%s/en_data.csv' % path, encoding='latin1')
  df = preprocess_dataframe(df)
  df = df[['i_dialog', 'i_utterance', 'speaker', 'utterance', 'context']]
  return df


train = read_json_make_dataframe('train')
print(train)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
mean length of sentence: 63.43404980588959
max length of sentence: 303
std dev length of sentence: 49.02061864789282
       i_dialog  ...    l
0             0  ...    1
1             0  ...    7
2             0  ...   13
3             0  ...   22
4             0  ...   27
...         ...  ...  ...
10556       719  ...  129
10557       719  ...  139
10558       719  ...  146
10559       719  ...  147
10560       719  ...  160

[10561 rows x 8 columns]


annotaion 변환

In [ ]:
# annotation 값을 emotion 값으로 변환함
def annotation_to_emotion(annotation):
    # 반올림 기반 변환법

    # 5 곱해서 반올림 친 벡터를 만듦
    rounded = [round(a * 5) for a in annotation]    

    # non-neutral check
    # 최대치가 같은게 두개 이상 있으면 neutral
    max_value = max(rounded)
    if len([x for x in rounded if x == max_value]) >= 2:
      return "non-neutral"
    
    # 맥스 밸류 인덱스에 해당하는 이모션 리턴
    return ['neutral','joy', 'sadness', 'fear', 'anger', 'surprise', 'disgust'][rounded.index(max_value)]

def summary_y_hats(y_hats):
  # y_hats의 분포를 보여주는 함수
  count_dict = {}
  
  for i in range(0, len(y_hats)):
    y_hat = y_hats[i]
    prediected_emotion = annotation_to_emotion(y_hat) # 예측한 이모션 값
    count_dict[prediected_emotion] = count_dict.get(prediected_emotion, 0) + 1

  print("summary of y:", count_dict)
  return
  

러닝 프로세스

In [ ]:
# 예측
def predict(data):
  from sklearn.metrics import f1_score, accuracy_score
  count = len(data)
  X = generate_X(data)
  y = generate_Y(data) # 정답
  y_hats = model.predict(X) # 예측치(annotation 벡터)

  print("")
  print("summary of y")
  summary_y_hats(y)

  print("")
  print("summary of y_hats")
  summary_y_hats(y_hats)

  y_true = [x for x in data['emotion'].values]
  y_pred = [annotation_to_emotion(x) for x in y_hats]

  acc = accuracy_score(y_true, y_pred)
  print("accuray: ", acc)
  print("f1_score: ",f1_score(y_true, y_pred, average='macro'))
  return acc

def run(index=""):
  data = read_csv_make_dataframe()

  count = len(data)
  X = generate_X(data)
  y_hats = model.predict(X) # 예측치(annotation 벡터)

  results = []
  columns = ['Id', 'Predicted']
  for i in range(0, count):
    y_hat = y_hats[i]
    prediected_emotion = annotation_to_emotion(y_hat) # 예측한 이모션 값

    results.append((i, prediected_emotion))

  pd.DataFrame(results, columns=columns).to_csv('%s/nlp_en_bert_%s.csv' % (path, index), index=False)
  return

def learn():
  # 테스트 데이터 읽음
  test = read_json_make_dataframe('test')
  acc = predict(test)
  accuracies = []
  for i in range(0, 1):
    epochs = 3
    batch_size = 32

    history = model.fit(X_train, 
                        y_train, 
                        epochs=epochs, 
                        batch_size=batch_size
    )
    acc = predict(test)
    accuracies.append(acc)
    learned = epochs * (i+1)
    run(learned)

  print("seq of accuracies:", accuracies)


라이브러리 설치

In [ ]:
!pip install keras-bert
!pip install keras-rectified-adam


모델 생성


In [ ]:
import codecs
import tensorflow as tf
from tqdm import tqdm
from chardet import detect
import keras
from keras_radam import RAdam
from keras import backend as K
from keras_bert import load_trained_model_from_checkpoint
# from google.colab import drive

SEQ_LEN = 80
LR = 2e-4


pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

try:
  model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
        seq_len=SEQ_LEN,
    )
except:
  !wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
  !unzip -o uncased_L-12_H-768_A-12.zip

  model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
        seq_len=SEQ_LEN,
    )


inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=7, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)


compiled = model.compile(
  RAdam(learning_rate =LR),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)


#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 80)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 80)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 80, 768), (3 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 80, 768)      1536        Input-Segment[0][0]              
____________________________________________________________________________________________

러닝 데이터

In [ ]:
# these sentences aren't that long so we may as well use the whole string
sequence_length = SEQ_LEN # 이 길이를 넘은 문장은 걍 짤림
max_features = 20000 # this is the number of words we care about

import codecs
from keras_bert import Tokenizer
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

tokenizer = Tokenizer(token_dict)
# X 데이터를 만들어줌
def generate_X(df):
  global tokenizer
  indices = []
  for utterance in df['utterance'].values:
    ids, segments = tokenizer.encode(utterance, max_len=SEQ_LEN)
    indices.append(ids)
  
  indices = np.array(indices)
  return [indices, np.zeros_like(indices)]


# Y 데이터를 만들어줌
def generate_Y(df):
  annotations = df['annotation'].apply(lambda x: pd.Series([float(c)/5 for c in x], dtype='float32'))
  # 어노테이션 값을 벡터화함.
  # 2120000 ->벡터화-> [2, 1, 2, 0, 0, 0, 0] ->나누기 5-> [0.4, 0.2, 0.4, 0, 0, 0, 0]
  return annotations.values

# X 데이터를 만들어줌
def generate_train_xy(df):
  x = generate_X(df)
  y = generate_Y(df)
  
  indices = x[0]
  mod = indices.shape[0] % 32
  if mod > 0:
      indices = indices[:-mod]

  return [indices, np.zeros_like(indices)], y[:-mod]


X_train, y_train = generate_train_xy(train)


실행


In [ ]:
learn()

mean length of sentence: 60.91280752532562
max length of sentence: 286
std dev length of sentence: 47.66618446797404

summary of y
summary of y: {'surprise': 327, 'neutral': 1369, 'joy': 320, 'non-neutral': 321, 'anger': 190, 'sadness': 105, 'fear': 47, 'disgust': 85}

summary of y_hats
summary of y: {'neutral': 1130, 'non-neutral': 1502, 'sadness': 128, 'joy': 4}
accuray:  0.32850940665701883
f1_score:  0.09980688268021566


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
10560/10560 [==============================] - 351s 33ms/step - loss: 1.8143 - accuracy: 0.4683
Epoch 2/3
10560/10560 [==============================] - 340s 32ms/step - loss: 1.8141 - accuracy: 0.4667
Epoch 3/3
10560/10560 [==============================] - 340s 32ms/step - loss: 1.8165 - accuracy: 0.4669

summary of y
summary of y: {'surprise': 327, 'neutral': 1369, 'joy': 320, 'non-neutral': 321, 'anger': 190, 'sadness': 105, 'fear': 47, 'disgust': 85}

summary of y_hats
summary of y: {'neutral': 1191, 'non-neutral': 1445, 'sadness': 124, 'joy': 4}
accuray:  0.3357452966714906
f1_score:  0.10114689021699111
mean length of sentence: 58.89381067961165
max length of sentence: 215
std dev length of sentence: 45.748533592551894
seq of accuracies: [0.3357452966714906]
